In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)
import timeit

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes
# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Tue 04 Apr 2023 03:44:33 PM PDT
ndbs
Linux gen-slurm-sexec-p0001 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/big_data_tests


# Find data

Run of 18 seeds, reflecting a population of 18 million, from Rajan on 3/17/2023:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19
```

In [2]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/v1.2_parallelize_business_names'
    '/united_states_of_america/2023_03_16_17_21_09'
    '/final_results/2023_03_17_15_28_19'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 44K
drwxrwsr-x  5 rmudambi IHME-Simulationscience 2.0K Mar 17 16:06 ..
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 16:04 tax_dependents_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:59 tax_1040_observer
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Mar 17 15:53 .
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:51 tax_w2_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:50 social_security_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 wic_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_cps
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_acs
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:48 decennial_census_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience  18K Mar 17 15:29 logs


In [3]:
!ls -halt $output_dir/tax_w2_observer

total 12G
-rw-r--r--  1 rmudambi IHME-Simulationscience 669M Mar 17 15:58 tax_w2_observer_6817.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 665M Mar 17 15:58 tax_w2_observer_4344.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 670M Mar 17 15:58 tax_w2_observer_3541.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 679M Mar 17 15:57 tax_w2_observer_4507.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 674M Mar 17 15:57 tax_w2_observer_6545.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_781.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 677M Mar 17 15:57 tax_w2_observer_9672.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_8869.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 675M Mar 17 15:57 tax_w2_observer_6810.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_1362.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 682M Mar 17 15:

# Check how much space full data takes up

24 GiB = 25.77 GB

In [4]:
# Total of 24 GiB in all subdirectories
!du -sh $output_dir

24G	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [5]:
# 24 GiB = 25.77 GB
2**30/1e9 * 24

25.769803776000003

# Load files using Categoricals, then save as HDF, time the steps, and see total disk usage

Altogether it took 4h 6min 52s to read the CSVs, convert datatypes to categoricals and integers, concatenate the shards for each observer, and save to HDF. The resulting HDF files take up 15GiB on disk, which is 62.5% of the 24GiB taken up by the CSV files.

The majority of the time went into reading in the `.csv`s and converting the dtypes to categorical and ints. This ranged from approximately 6.3s for the ACS observer to approximately 78 minutes for the W2 observer, and it took about 13.5 min for the decennial census.

I used `complib='bzip2'` and  `complevel=9` when saving the HDFs. The time to save the HDFs ranged from 320 ms for the ACS observer to 29min 37s for the W2 observer. The decennial census HDF took 3min 56s to save.

**Note:** I added a few lines of code in the cell below after running it, namely the `del` statements and the calculation of memory usage, so this will not be reflected in the output.

In [6]:
save_dir = '/share/scratch/users/ndbs/prl/2023_03_16_17_21_09'
!ls -halt $save_dir

total 8.0K
drwxrwxr-x 2 ndbs Domain Users    0 Apr  4 15:44 .
drwxrwxr-x 6 ndbs Domain Users 2.5K Apr  4 15:26 ..


In [7]:
%%time
observer_names = []
nrows = None
ext = '.csv.bz2'
seed_length=4
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    print("="*50)
    print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
#             seed = filename_root[:-seed_length]
            filepath = os.path.join(dirpath, filename)
            %time shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
    if shards:
        # Before concatenating the dataframes, we need to make sure they have the same categories
        # in each categorical column, so take the union of categories across shards
        print("\nSetting categories for", observer_name)
        categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
        for col in categorical_cols:
            categories = shards[0][col].cat.categories
            for shard in shards[1:]:
                categories = categories.union(shard[col].cat.categories)
            for shard in shards:
                shard[col] = shard[col].cat.set_categories(categories)
        print("Concatenating data for", observer_name)
        %time df = pd.concat(shards, ignore_index=True)
        print("Calculating memory usage")
        %time print(sizemb(df), "MB")
        observer_names.append(observer_name)
        print("Saving data for", observer_name)
        %time df.to_hdf( \
            f'{save_dir}/{observer_name}.hdf', key=observer_name, \
            format='table', complib='bzip2', complevel=9, \
        )
        # Not sure if I need this, but we can delete dataframes to save memory
        del df
        del shards
print('\nObservers:', observer_names, '\n')
!ls -halt $save_dir

Loading data from 2023_03_17_15_28_19 

Loading data from decennial_census_observer 

CPU times: user 38 s, sys: 1.21 s, total: 39.2 s
Wall time: 39.3 s
CPU times: user 40 s, sys: 947 ms, total: 40.9 s
Wall time: 41 s
CPU times: user 39.9 s, sys: 811 ms, total: 40.7 s
Wall time: 40.8 s
CPU times: user 41 s, sys: 723 ms, total: 41.7 s
Wall time: 41.7 s
CPU times: user 41 s, sys: 668 ms, total: 41.6 s
Wall time: 41.7 s
CPU times: user 42.1 s, sys: 712 ms, total: 42.9 s
Wall time: 42.9 s
CPU times: user 42.6 s, sys: 696 ms, total: 43.3 s
Wall time: 43.4 s
CPU times: user 42.6 s, sys: 704 ms, total: 43.3 s
Wall time: 43.3 s
CPU times: user 43 s, sys: 645 ms, total: 43.7 s
Wall time: 43.7 s
CPU times: user 43.3 s, sys: 656 ms, total: 43.9 s
Wall time: 44 s
CPU times: user 42.1 s, sys: 792 ms, total: 42.9 s
Wall time: 43 s
CPU times: user 44.7 s, sys: 587 ms, total: 45.2 s
Wall time: 45.3 s
CPU times: user 44.7 s, sys: 680 ms, total: 45.3 s
Wall time: 45.4 s
CPU times: user 44.6 s, sys: 552 

CPU times: user 2.17 s, sys: 8.08 ms, total: 2.18 s
Wall time: 2.19 s
CPU times: user 1.88 s, sys: 7.92 ms, total: 1.89 s
Wall time: 1.9 s
CPU times: user 2.3 s, sys: 8 ms, total: 2.3 s
Wall time: 2.31 s
CPU times: user 1.92 s, sys: 15.9 ms, total: 1.93 s
Wall time: 1.94 s
CPU times: user 1.94 s, sys: 8.02 ms, total: 1.95 s
Wall time: 1.96 s
CPU times: user 2.1 s, sys: 3.94 ms, total: 2.1 s
Wall time: 2.11 s
CPU times: user 2.19 s, sys: 0 ns, total: 2.19 s
Wall time: 2.2 s
CPU times: user 2.65 s, sys: 3.91 ms, total: 2.66 s
Wall time: 2.67 s
CPU times: user 2.07 s, sys: 3.93 ms, total: 2.07 s
Wall time: 2.08 s
CPU times: user 2.44 s, sys: 4.01 ms, total: 2.44 s
Wall time: 2.45 s
CPU times: user 2.1 s, sys: 3.81 ms, total: 2.11 s
Wall time: 2.12 s
CPU times: user 1.9 s, sys: 28 µs, total: 1.9 s
Wall time: 1.9 s
CPU times: user 2 s, sys: 4.1 ms, total: 2 s
Wall time: 2.01 s
CPU times: user 2.1 s, sys: 12 ms, total: 2.11 s
Wall time: 2.12 s

Setting categories and concatenating data for  

# Read data back in, time it, and see how much space everything takes up

It took 24min 27s to read in all the data, and it takes up about 38.5 GB = 35.9 GiB in memory.

The read time ranged from 1.22 s for the ACS data to 11min 42s for the W2 data. The decennial census took 1min 25s.

The memory usage ranged from 54 MB = 51.5 MiB for the ACS data to 16 GB = 14.9 GiB for the W2 data.

I don't know how much _extra_ memory it takes to read in the `.hdf` file beyond the memory usage of the final DataFrame.

In [8]:
%%time
dfs = {}
ext = '.hdf'
for dirpath, dirnames, filenames in os.walk(save_dir):
    print(dirpath, dirnames, filenames)
    for filename in filenames:
        if filename.endswith(ext):
            observer_name = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            print("\nLoading data from", observer_name)
            %time dfs[observer_name] = pd.read_hdf(filepath)
            print("Computing memory usage")
            %time print(sizemb(dfs[observer_name]), 'MB')

dfs = MappingViaAttributes(dfs)
print(dfs.keylist())
print("\nTotal memory usage")
%time print(sum(sizemb(df) for df in dfs.values()), "MB")
print("\nTotal time")

/share/scratch/users/ndbs/prl/2023_03_16_17_21_09 [] ['tax_1040_observer.hdf', 'wic_observer.hdf', 'tax_dependents_observer.hdf', 'decennial_census_observer.hdf', 'tax_w2_observer.hdf', 'social_security_observer.hdf', 'household_survey_observer_acs.hdf', 'household_survey_observer_cps.hdf']

Loading data from tax_1040_observer
CPU times: user 7min 43s, sys: 12.3 s, total: 7min 55s
Wall time: 8min 3s
Computing memory usage
13370.361975 MB
CPU times: user 311 ms, sys: 60 ms, total: 371 ms
Wall time: 370 ms

Loading data from wic_observer
CPU times: user 7.35 s, sys: 96.4 ms, total: 7.45 s
Wall time: 7.54 s
Computing memory usage
293.823706 MB
CPU times: user 89.7 ms, sys: 32 µs, total: 89.7 ms
Wall time: 89.6 ms

Loading data from tax_dependents_observer
CPU times: user 2min 32s, sys: 3.7 s, total: 2min 35s
Wall time: 2min 38s
Computing memory usage
4655.984061 MB
CPU times: user 196 ms, sys: 28 ms, total: 224 ms
Wall time: 224 ms

Loading data from decennial_census_observer
CPU times: u

# Look at the data and datatypes

## Decennial Census

37.07 million rows

In [9]:
dfs.decennial_census_observer

,first_name,relation_to_household_head,guardian_1,zipcode,street_name,state,guardian_2,race_ethnicity,unit_number,simulant_id,housing_type,last_name,middle_initial,city,date_of_birth,street_number,sex,age,year
0,Herbert,Reference person,-1,93117.0,south 11th street,CA,-1,White,NaN,1362000000000,Standard,Redden,S,carmel,1958-12-16,8,Male,61.292465,2020
1,Emery,Biological child,1362000000000,93117.0,south 11th street,CA,-1,White,NaN,1362000000001,Standard,Redden,S,carmel,2012-09-18,8,Female,7.534608,2020
2,Flora,Roommate,-1,93117.0,south 11th street,CA,-1,White,NaN,1362000000002,Standard,Thomas,S,carmel,1982-02-13,8,Female,38.131340,2020
3,Emily,Reference person,-1,55408.0,old us 35 e,MN,-1,White,apt num 306,1362000000003,Standard,Norris,S,cottage grove,1985-08-07,1605,Female,34.650085,2020
4,Garrett,Opp-sex spouse,-1,55408.0,old us 35 e,MN,-1,White,apt num 306,1362000000004,Standard,Norris,S,cottage grove,1980-08-05,1605,Male,39.654461,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37078268,Meredith,Reference person,-1,77006.0,stoughton avenue,TX,-1,Multiracial or Other,NaN,8869001178220,Standard,Gallardo Riener,P,spring,1998-03-02,1285,Female,32.126720,2030
37078269,Andre,Opp-sex spouse,-1,77006.0,stoughton avenue,TX,-1,Multiracial or Other,NaN,8869001178221,Standard,Gallardo Riener,S,spring,1997-01-02,1285,Male,33.287312,2030
37078270,Hugh,Reference person,-1,27410.0,cardinal crest ct,NC,-1,White,NaN,8869001178222,Standard,Lewis,S,mooresville,1971-07-11,321,Male,58.766632,2030
37078271,Rosalyn,Opp-sex spouse,-1,27410.0,cardinal crest ct,NC,-1,White,NaN,8869001178223,Standard,Lewis,T,mooresville,1972-04-18,321,Female,57.995991,2030


In [10]:
dfs.decennial_census_observer.dtypes

first_name                    category
relation_to_household_head    category
guardian_1                       int64
zipcode                       category
street_name                   category
state                         category
guardian_2                       int64
race_ethnicity                category
unit_number                   category
simulant_id                      int64
housing_type                  category
last_name                     category
middle_initial                category
city                          category
date_of_birth                 category
street_number                 category
sex                           category
age                            float32
year                             int64
dtype: object

## W2 / 1099

175.56 million rows

In [11]:
dfs.tax_w2_observer

,mailing_address_street_number,mailing_address_state,employer_name,date_of_birth,employer_id,last_name,mailing_address_unit_number,first_name,is_w2,employer_state,...,mailing_address_city,employer_street_name,ssn,employer_street_number,income,mailing_address_street_name,middle_initial,age,tax_year,employer_city
0,2002,CT,Adore Hair & Field Kindergarten,1986-04-19,1550757,King,NaN,Bethany,True,NJ,...,enfield,edgecliff ct,703807490,NaN,26195.775391,pecan st,N,34.642536,2020,cherry hill
1,13306,KY,Grand Canyon Vista Hermosa Christian Church,1943-12-08,389376,Rector,NaN,Carolyn,True,FL,...,louisville,n 52nd st,467525395,309,793.475586,co rd 9 ne,O,77.003464,2020,fort myers
2,13306,KY,Town & Spa & deli and Culture Barbershop,1943-12-08,1354620,Rector,NaN,Carolyn,True,AZ,...,louisville,skyview ter,467525395,1960,5330.663574,co rd 9 ne,O,77.003464,2020,tucson
3,13306,KY,Mama C's Food,1943-12-08,1393914,Rector,NaN,Carolyn,True,WI,...,louisville,ince dr,467525395,e,2174.945068,co rd 9 ne,O,77.003464,2020,milwaukee
4,NaN,WV,Harding Manor Inn,1961-04-17,1535195,Engdahl,NaN,Cindy,True,GA,...,beckley,stoney crk cir,429869852,NaN,22593.068359,NaN,R,59.647953,2020,augusta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175587484,61,MA,Riverland Food Mart,2008-08-10,1422582,Ta,NaN,Ronald,True,AZ,...,chicopee,elliott ave,282722156,3,1494.691162,alyssum pl,C,21.453884,2029,sierra vista city
175587485,602,MS,Gordon Union,1955-05-26,772347,Kraus,NaN,Margaret,True,IA,...,long beach,e loretta ave,831240706,912,4404.092773,baldwin avenue,D,74.662674,2029,ely
175587486,11855,VA,Kilbourne High,1992-12-13,607405,Garcia,NaN,Alexander,True,IL,...,chesapeake,e 2200 n,717614657,1117,2048.272217,martin l king jr dr nw,Z,37.110645,2029,lebanon
175587487,2208,IL,Mitch Marcus,1989-01-02,220500,Maldonado,NaN,Evan,True,UT,...,johnsburg,w hampton av,595818272,NaN,4062.245850,adair rd,A,41.056873,2029,vernal


In [12]:
dfs.tax_w2_observer.dtypes

mailing_address_street_number    category
mailing_address_state            category
employer_name                    category
date_of_birth                    category
employer_id                         int64
last_name                        category
mailing_address_unit_number      category
first_name                       category
is_w2                                bool
employer_state                   category
mailing_address_zipcode          category
simulant_id                         int64
employer_unit_number             category
employer_zipcode                 category
mailing_address_po_box           category
mailing_address_city             category
employer_street_name             category
ssn                                 int32
employer_street_number           category
income                            float32
mailing_address_street_name      category
middle_initial                   category
age                               float32
tax_year                         c

## Social security data

22.21 million rows

In [13]:
dfs.social_security_observer

,event_type,date_of_birth,ssn,last_name,first_name,middle_initial,simulant_id,event_date
0,creation,1921-03-04,185777085,Kaysen,Gertrude,E,5616000453898,1921-03-04
1,creation,1921-03-04,481234888,Shanahan,Luther,R,5616000788874,1921-03-04
2,creation,1921-03-23,388411615,Johnson,Darrell,R,5616000170011,1921-03-23
3,creation,1921-04-01,367785835,Strauss,Colleen,M,5616000304292,1921-04-01
4,creation,1921-07-07,9218699,Lawrence,Dorothy,M,5616000917550,1921-07-07
...,...,...,...,...,...,...,...,...
22207910,death,2030-02-04,471389204,Goller,Mia,Y,2284001174592,2030-05-15
22207911,death,2030-04-02,12345596,Sun,Robert,T,2284001176964,2030-05-15
22207912,death,2030-04-13,488520040,Johnson,Matteo,M,2284001177661,2030-05-15
22207913,death,2030-04-14,135694273,Mullett,Myles,S,2284001177778,2030-05-15


In [14]:
dfs.social_security_observer.dtypes

event_type        category
date_of_birth     category
ssn                  int32
last_name         category
first_name        category
middle_initial    category
simulant_id          int64
event_date        category
dtype: object

## WIC administrative data

2.68 million rows

In [15]:
dfs.wic_observer

,housing_type,guardian_2,date_of_birth,year,state,guardian_1,middle_initial,sex,street_name,city,race_ethnicity,zipcode,last_name,simulant_id,relation_to_household_head,unit_number,age,household_id,street_number,first_name
0,Standard,-1,2006-01-25,2021,TX,-1,E,Female,marina pointe vlg ct,amarillo,Black,77706.0,Payne,781000000210,Reference person,NaN,14.871357,781000000094,3386,Emely
1,Standard,-1,1993-07-29,2021,MD,-1,G,Female,s sr 104,lanham,Black,21122.0,Frazier Dolton,781000000555,Reference person,NaN,27.363504,781000000235,NaN,Amy
2,Standard,-1,1984-01-12,2021,PA,-1,C,Female,windsor ct,philadelphia,White,19123.0,Ruszczyk,781000000563,Reference person,NaN,36.910057,781000000241,1915,Amie
3,Standard,-1,1998-07-20,2021,NE,-1,S,Female,ladue acres dr,bennington,White,68504.0,Hoagland Gentle,781000000629,Reference person,NaN,22.390255,781000000265,555,Jennifer
4,Standard,-1,1985-07-29,2021,WI,-1,A,Female,conley road se,black earth,White,54115.0,Duffield,781000000881,Opp-sex spouse,NaN,35.364902,781000000365,2773,Christina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679338,Standard,5020000595381,2030-01-07,2030,CA,5020000595382,J,Male,northeast 9th plc,alhambra,Latino,92028.0,Pacheco,5020001173735,Biological child,NaN,0.044600,5020000239404,1144,Camden
2679339,Standard,-1,2029-12-27,2030,CA,5020000618658,M,Female,ruthwell dr,palos verdes peninsula,Latino,95838.0,Davis Guzman,5020001173736,Other nonrelative,NaN,0.074165,5020000071420,992,Charlotte
2679340,Standard,-1,2029-12-30,2030,TX,5020000727232,H,Male,spirea rd,rockwall,Asian,75550.0,Judge,5020001173741,Biological child,NaN,0.067143,5020000457474,664,Terrance
2679341,Standard,-1,2030-01-19,2030,CO,5020000939518,A,Male,fm 66,arvada,White,80439.0,Bulman,5020001173745,Biological child,NaN,0.011262,5020000408935,17019,Kamari


In [16]:
dfs.wic_observer.dtypes

housing_type                  category
guardian_2                       int64
date_of_birth                 category
year                             int64
state                         category
guardian_1                       int64
middle_initial                category
sex                           category
street_name                   category
city                          category
race_ethnicity                category
zipcode                       category
last_name                     category
simulant_id                      int64
relation_to_household_head    category
unit_number                   category
age                            float32
household_id                     int64
street_number                 category
first_name                    category
dtype: object

## ACS household survey

0.37 million rows

In [17]:
dfs.household_survey_observer_acs

,guardian_2,zipcode,guardian_1,household_id,state,sex,date_of_birth,city,last_name,first_name,street_name,simulant_id,unit_number,street_number,survey_date,housing_type,age,middle_initial
0,-1,45030.0,-1,5616000000740,OH,Female,1940-02-02,dublin,Endsley,Gladys,mount lubentia ct w,5616000001859,NaN,817,2020-04-29,Standard,80.160934,E
1,-1,45030.0,-1,5616000000740,OH,Female,1945-11-28,dublin,Brown,Sharon,mount lubentia ct w,5616000001860,NaN,817,2020-04-29,Standard,74.339783,B
2,-1,30274.0,-1,5616000011949,GA,Female,1982-08-21,lawrenceville,Frazier,Cheree,greenwood avenue north,5616000029767,NaN,300,2020-04-29,Standard,37.612480,A
3,-1,30274.0,-1,5616000011949,GA,Male,1998-01-21,lawrenceville,Frazier,Nathan,greenwood avenue north,5616000029768,NaN,300,2020-04-29,Standard,22.191723,M
4,-1,30274.0,5616000029767,5616000011949,GA,Female,2003-08-18,lawrenceville,Frazier,Hannah,greenwood avenue north,5616000029769,NaN,300,2020-04-29,Standard,16.622293,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369637,-1,68154.0,3167000689704,3167000277625,NE,Male,2025-12-27,omaha,Adsitt,Maverick,harmon av,3167001100106,NaN,510,2030-05-15,Standard,4.381575,L
369638,-1,37308.0,3167000814381,3167000327705,TN,Male,2028-06-08,clarksville,Garcia Faustino,Declan,ovlk wy,3167001143670,NaN,3248,2030-05-15,Standard,1.934982,L
369639,-1,84061.0,-1,3167000485611,UT,Male,1989-10-17,spanish frk,Roles,Brandon,e 115 st,3167001152319,apt 1,6624,2030-05-15,Standard,40.575119,C
369640,3167000377594,6371.0,3167000377595,3167000152170,CT,Female,2029-10-03,stamford,Rizzo,Ayla,victoria ave,3167001168140,NaN,20471,2030-05-15,Standard,0.613333,M


In [18]:
dfs.household_survey_observer_acs.dtypes

guardian_2           int64
zipcode           category
guardian_1           int64
household_id         int64
state             category
sex               category
date_of_birth     category
city              category
last_name         category
first_name        category
street_name       category
simulant_id          int64
unit_number       category
street_number     category
survey_date       category
housing_type      category
age                float32
middle_initial    category
dtype: object

## CPS household survey

1.93 million rows

In [19]:
dfs.household_survey_observer_cps

,housing_type,state,street_name,age,survey_date,zipcode,street_number,guardian_2,date_of_birth,household_id,simulant_id,first_name,guardian_1,sex,unit_number,city,middle_initial,last_name
0,Standard,IL,pinata pl ne,90.283279,2020-04-29,60518.0,803,-1,1929-12-19,4507000001122,4507000002737,Peggy,-1,Female,NaN,chicago,B,Snapp
1,Standard,NM,chaparral ave,75.797470,2020-04-29,87507.0,210,-1,1944-06-14,4507000001145,4507000002801,Dave,-1,Male,NaN,albuquerque,C,Charnesky
2,Standard,NM,chaparral ave,68.601082,2020-04-29,87507.0,210,-1,1951-08-25,4507000001145,4507000002802,Betty,-1,Female,NaN,albuquerque,D,Charnesky
3,Standard,MO,francis ave,29.933657,2020-04-29,63115.0,850,-1,1990-04-26,4507000005727,4507000014212,Lauren,-1,Female,NaN,kansas-city,J,Bonvouloir
4,Standard,WA,frm-to-market road 858,80.027611,2020-04-29,98122.0,432,-1,1940-03-22,4507000006298,4507000015620,Connie,-1,Female,NaN,bellevue,M,Cooper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928125,Standard,MD,laurel grove dr,17.694689,2030-05-15,21804.0,10,-1,2012-09-03,781000607093,781001177710,Eunice,-1,Female,NaN,rosedale,J,Perez
1928126,Standard,MD,laurel grove dr,20.090551,2030-05-15,21804.0,10,-1,2010-04-12,781000607093,781001177711,Grayson,-1,Male,NaN,rosedale,C,Perez
1928127,Standard,MD,laurel grove dr,8.753614,2030-05-15,21804.0,10,781001177711,2021-08-13,781000607093,781001177712,Hayden,781001177710,Female,NaN,rosedale,X,Perez
1928128,Standard,MD,laurel grove dr,4.181302,2030-05-15,21804.0,10,781001177711,2026-03-10,781000607093,781001177713,Audrey,781001177710,Female,NaN,rosedale,A,Perez


In [20]:
dfs.household_survey_observer_cps.dtypes

housing_type      category
state             category
street_name       category
age                float32
survey_date       category
zipcode           category
street_number     category
guardian_2           int64
date_of_birth     category
household_id         int64
simulant_id          int64
first_name        category
guardian_1           int64
sex               category
unit_number       category
city              category
middle_initial    category
last_name         category
dtype: object

## 1040 observer

215.72 million rows

In [21]:
dfs.tax_1040_observer

,tax_year,mailing_address_po_box,itin,mailing_address_city,date_of_birth,first_name,mailing_address_street_number,mailing_address_zipcode,middle_initial,ssn,simulant_id,housing_type,mailing_address_unit_number,mailing_address_street_name,joint_filer,age,mailing_address_state,last_name
0,2019,0,-1,unincorporated,1961-12-21,Brenda,440,34238.0,J,699629565,6810000000000,Standard,NaN,sabre hill dr,False,58.277176,FL,Blair
1,2019,0,-1,unincorporated,1959-10-16,Carl,440,34238.0,J,558284736,6810000000001,Standard,NaN,sabre hill dr,True,60.460503,FL,Blair
2,2019,0,-1,unincorporated,1996-06-12,Armand,440,34238.0,H,736879950,6810000000002,Standard,NaN,sabre hill dr,False,23.802605,FL,Blair
3,2019,0,-1,unincorporated,1999-01-29,Armand,440,34238.0,H,652321967,6810000000003,Standard,NaN,sabre hill dr,False,21.173023,FL,Blair
4,2019,0,-1,baldwin park,1966-04-14,Bruce,3078,93551.0,J,132771940,6810000000004,Standard,NaN,sunnyside av,False,53.964569,CA,Stile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215718830,2029,0,-1,toppenish,1998-01-15,Cole,5,98121.0,D,766229633,5949001178587,Standard,NaN,melody cir,False,32.251614,WA,Chee
215718831,2029,0,-1,toppenish,2000-01-01,Emily,5,98121.0,V,477426009,5949001178588,Standard,NaN,melody cir,True,30.291912,WA,Chee
215718832,2029,0,908737710,lake panasoffkee,1967-06-17,Paula,8379,33610.0,C,-1,5949001178589,Standard,NaN,wexford dr,False,62.833714,FL,Ibarra
215718833,2029,0,-1,lake panasoffkee,1967-07-25,Frankie,8379,33610.0,R,461942366,5949001178590,Standard,NaN,wexford dr,True,62.730888,FL,Ibarra


In [22]:
dfs.tax_1040_observer.dtypes

tax_year                         category
mailing_address_po_box           category
itin                                int32
mailing_address_city             category
date_of_birth                    category
first_name                       category
mailing_address_street_number    category
mailing_address_zipcode          category
middle_initial                   category
ssn                                 int32
simulant_id                         int64
housing_type                     category
mailing_address_unit_number      category
mailing_address_street_name      category
joint_filer                          bool
age                               float32
mailing_address_state            category
last_name                        category
dtype: object

## Tax dependents observer

71.37 million rows

In [23]:
dfs.tax_dependents_observer

,ssn,mailing_address_street_number,mailing_address_unit_number,mailing_address_zipcode,mailing_address_city,mailing_address_po_box,housing_type,middle_initial,mailing_address_state,first_name,last_name,sex,guardian_id,tax_year,age,simulant_id,mailing_address_street_name,date_of_birth
0,444320195,719,NaN,6902.0,waterbury,0,Standard,E,CT,Camden,Jackson,Male,3254000000005,2020,13.353219,3254000000006,poplar st,2007-08-03
1,52880129,3247,NaN,21211.0,frederick,0,Standard,E,MD,Camren,Jackson,Male,3254000000005,2020,7.245981,3254000000007,cogliandro dr,2013-09-10
2,91798969,3247,NaN,21211.0,frederick,0,Standard,E,MD,Carson,Jackson,Male,3254000000005,2020,4.541136,3254000000008,cogliandro dr,2016-05-25
3,110158368,402,NaN,21215.0,loch raven,0,Standard,E,MD,Breanna,Wade,Female,3254000000012,2020,17.116179,3254000000013,s indian wells dr,2003-10-28
4,459204723,228,NaN,76502.0,jacksonville,0,Standard,E,TX,Bianca,Tufts,Female,3254000000016,2020,15.220131,3254000000018,village rd,2005-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71372774,766454650,14865,NaN,76262.0,granite shoals,0,Standard,S,TX,Willow,Diazlopez,Female,3167001173228,2029,4.154457,3167001173231,custer st,2025-11-28
71372775,857568795,24302,NaN,78653.0,austin,0,Standard,Z,TX,Amiyah,Hernandez,Female,3167001173236,2029,7.325635,3167001173237,budworth circle,2022-09-26
71372776,455047998,24302,NaN,78653.0,austin,0,Standard,C,TX,Axel,Hernandez,Male,3167001173236,2029,3.686220,3167001173238,budworth circle,2026-05-18
71372777,186812653,700,NaN,66210.0,topeka,0,Standard,V,KS,Lily,Logan,Female,3167001173240,2029,13.622475,3167001173241,norstead walk,2016-06-09


In [24]:
dfs.tax_dependents_observer.dtypes

ssn                                 int32
mailing_address_street_number    category
mailing_address_unit_number      category
mailing_address_zipcode          category
mailing_address_city             category
mailing_address_po_box           category
housing_type                     category
middle_initial                   category
mailing_address_state            category
first_name                       category
last_name                        category
sex                              category
guardian_id                         int64
tax_year                         category
age                               float32
simulant_id                         int64
mailing_address_street_name      category
date_of_birth                    category
dtype: object